In [1]:
sc.version

'2.4.5'

In [18]:
#from pyspark import SparkContext
#sc = SparkContext.getOrCreate()
#sqlContext = SQLContext(sc)

df = sqlContext.read.format('csv').options(header='true', inferschema='true')\
    .load('../FALLECIDOS.csv')
df.rdd.take(3)

[Row(UUID='056f73432adaf077b06ab0f9453ce753', FECHA_FALLECIMIENTO='24/03/2020', SEXO='MASCULINO', FECHA_NAC=None, DEPARTAMENTO='CUSCO', PROVINCIA='CUSCO', DISTRITO='CUSCO'),
 Row(UUID='4bac0872ed7858eec6b438947227cf66', FECHA_FALLECIMIENTO='27/03/2020', SEXO='MASCULINO', FECHA_NAC=None, DEPARTAMENTO='CUSCO', PROVINCIA='CUSCO', DISTRITO='CUSCO'),
 Row(UUID='27aa0530a6faa06b6589a9271853f66d', FECHA_FALLECIMIENTO='9/05/2020', SEXO='FEMENINO', FECHA_NAC=None, DEPARTAMENTO='TUMBES', PROVINCIA='TUMBES', DISTRITO='PAMPAS DE HOSPITAL')]

In [19]:
#Feature selection
df2=df.select('FECHA_FALLECIMIENTO','SEXO','FECHA_NAC','DEPARTAMENTO','PROVINCIA','DISTRITO')
#df2.show()

In [23]:
drop_list = ['UUID'] # Remove Id column
df2 = df.select([column for column in df.columns if column not in drop_list])
df2.show()

+-------------------+---------+---------+------------+---------+------------------+
|FECHA_FALLECIMIENTO|     SEXO|FECHA_NAC|DEPARTAMENTO|PROVINCIA|          DISTRITO|
+-------------------+---------+---------+------------+---------+------------------+
|         24/03/2020|MASCULINO|     null|       CUSCO|    CUSCO|             CUSCO|
|         27/03/2020|MASCULINO|     null|       CUSCO|    CUSCO|             CUSCO|
|          9/05/2020| FEMENINO|     null|      TUMBES|   TUMBES|PAMPAS DE HOSPITAL|
|         30/03/2020|MASCULINO|     null|        LIMA|     LIMA|        SAN ISIDRO|
|          2/04/2020|MASCULINO|     null|       CUSCO|    CUSCO|             CUSCO|
|          6/04/2020|MASCULINO|     null|        LIMA|     null|              null|
|         16/04/2020|MASCULINO|     null|      LORETO|     null|              null|
|         17/04/2020| FEMENINO|     null|      LORETO|     null|              null|
|         18/04/2020|MASCULINO|     null|        LIMA|     null|            

In [24]:
df2.printSchema()

root
 |-- FECHA_FALLECIMIENTO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- FECHA_NAC: string (nullable = true)
 |-- DEPARTAMENTO: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- DISTRITO: string (nullable = true)



In [31]:
df2.groupBy("DEPARTAMENTO").count().show()

+-------------+-----+
| DEPARTAMENTO|count|
+-------------+-----+
|     AYACUCHO|    3|
|       CALLAO|  194|
|        PASCO|    8|
|  LA LIBERTAD|   93|
|     AMAZONAS|   18|
|      UCAYALI|  104|
|       TUMBES|   68|
|        JUNIN|   14|
|     AREQUIPA|   25|
|         PUNO|    6|
|   LAMBAYEQUE|  566|
|MADRE DE DIOS|    6|
|      HUANUCO|    7|
|        PIURA|  429|
| HUANCAVELICA|    1|
|       LORETO|  276|
|    CAJAMARCA|    3|
|     MOQUEGUA|    1|
|   SAN MARTIN|   14|
|        TACNA|    4|
+-------------+-----+
only showing top 20 rows



In [32]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

stringIndexer = StringIndexer(inputCol = "DEPARTAMENTO", outputCol = "label")

# Solo para mostrar
modelx = stringIndexer.fit(df2)
indexedx = modelx.transform(df2)
#type(modelx)

In [33]:
indexedx.toPandas()

,FECHA_FALLECIMIENTO,SEXO,FECHA_NAC,DEPARTAMENTO,PROVINCIA,DISTRITO,label
0,24/03/2020,MASCULINO,None,CUSCO,CUSCO,CUSCO,21.0
1,27/03/2020,MASCULINO,None,CUSCO,CUSCO,CUSCO,21.0
2,9/05/2020,FEMENINO,None,TUMBES,TUMBES,PAMPAS DE HOSPITAL,9.0
3,30/03/2020,MASCULINO,None,LIMA,LIMA,SAN ISIDRO,0.0
4,2/04/2020,MASCULINO,None,CUSCO,CUSCO,CUSCO,21.0
...,...,...,...,...,...,...,...
3451,17/04/2020,MASCULINO,16/02/1952,LIMA,LIMA,VILLA MARIA DEL TRIUNFO,0.0
3452,7/04/2020,MASCULINO,18/10/1981,LIMA,LIMA,VILLA MARIA DEL TRIUNFO,0.0
3453,21/05/2020,MASCULINO,6/04/1956,LORETO,NULL,NULL,3.0
3454,13/05/2020,MASCULINO,None,LORETO,NULL,NULL,3.0


In [48]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=[], outputCol="features")

dfa = vecAssembler.transform(df2) # this will add to the table outputCol column with vectors.
dfa.show()# dfa ya no es necesario, ahora se encarga Pipeline

+-------------------+---------+---------+------------+---------+------------------+--------+
|FECHA_FALLECIMIENTO|     SEXO|FECHA_NAC|DEPARTAMENTO|PROVINCIA|          DISTRITO|features|
+-------------------+---------+---------+------------+---------+------------------+--------+
|         24/03/2020|MASCULINO|     null|       CUSCO|    CUSCO|             CUSCO|      []|
|         27/03/2020|MASCULINO|     null|       CUSCO|    CUSCO|             CUSCO|      []|
|          9/05/2020| FEMENINO|     null|      TUMBES|   TUMBES|PAMPAS DE HOSPITAL|      []|
|         30/03/2020|MASCULINO|     null|        LIMA|     LIMA|        SAN ISIDRO|      []|
|          2/04/2020|MASCULINO|     null|       CUSCO|    CUSCO|             CUSCO|      []|
|          6/04/2020|MASCULINO|     null|        LIMA|     null|              null|      []|
|         16/04/2020|MASCULINO|     null|      LORETO|     null|              null|      []|
|         17/04/2020| FEMENINO|     null|      LORETO|     null|      

In [39]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vecAssembler, stringIndexer])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df2)
dataset = pipelineFit.transform(df2)
dataset.show(5)

IllegalArgumentException: 'Field "." does not exist.\nAvailable fields: FECHA_FALLECIMIENTO, SEXO, FECHA_NAC, DEPARTAMENTO, PROVINCIA, DISTRITO'

In [43]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

NameError: name 'dataset' is not defined

In [14]:
#trainingData.show()

In [15]:
#testData.show()

In [16]:
from pyspark.ml.classification import NaiveBayes
